Launch local S3 via s3proxy in a docker container (will not persist data):

```
docker run -p 9000:80 --env S3PROXY_AUTHORIZATION=none andrewgaul/s3proxy
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import boto3, botocore
from fs import open_fs
import os
import sys

from jupyterfs.pyfilesystem_manager import PyFilesystemContentsManager

## Create a bucket to use as the filesystem

In [3]:
bucket_name = 'foo'
boto_kwargs = dict(
    config=botocore.client.Config(signature_version=botocore.UNSIGNED),
    endpoint_url='http://127.0.0.1:9000',
#     aws_access_key_id='',
#     aws_secret_access_key='',
#     region_name=self.region,
)

client = boto3.client('s3', **boto_kwargs)
resource = boto3.resource('s3', **boto_kwargs)

# check if bucket already exists
bucket = resource.Bucket(bucket_name)
bucket_exists = True
try:
    resource.meta.client.head_bucket(Bucket=bucket_name)
except botocore.exceptions.ClientError as e:
    # If it was a 404 error, then the bucket does not exist.
    error_code = e.response['Error']['Code']
    if error_code == '404':
        bucket_exists = False

if bucket_exists:
    # dump info on any existing s3 contents
    for key in bucket.objects.all():
        print(key)
    print(client.list_objects(Bucket=bucket_name))
    print(client.list_objects_v2(Bucket=bucket_name))

    # delete the bucket
    for key in bucket.objects.all():
        key.delete()
    bucket.delete()

client.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': '4442587FB7D0A2F9', 'HostId': '', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 20 Mar 2020 07:23:24 GMT', 'x-amz-request-id': '4442587FB7D0A2F9', 'content-type': 'application/xml; charset=UTF-8', 'transfer-encoding': 'chunked', 'server': 'Jetty(9.2.z-SNAPSHOT)'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Marker': '', 'Name': 'foo', 'Prefix': '', 'MaxKeys': 1000, 'EncodingType': 'url'}
{'ResponseMetadata': {'RequestId': '4442587FB7D0A2F9', 'HostId': '', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 20 Mar 2020 07:23:24 GMT', 'x-amz-request-id': '4442587FB7D0A2F9', 'content-type': 'application/xml; charset=UTF-8', 'transfer-encoding': 'chunked', 'server': 'Jetty(9.2.z-SNAPSHOT)'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Name': 'foo', 'Prefix': '', 'MaxKeys': 1000, 'EncodingType': 'url', 'KeyCount': 0, 'ContinuationToken': '', 'StartAfter': ''}


{'ResponseMetadata': {'RequestId': '4442587FB7D0A2F9',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 20 Mar 2020 07:23:24 GMT',
   'x-amz-request-id': '4442587FB7D0A2F9',
   'location': '/foo',
   'content-length': '0',
   'server': 'Jetty(9.2.z-SNAPSHOT)'},
  'RetryAttempts': 0},
 'Location': '/foo'}

## Set up a S3 PyFilesystemContentsManager

In [4]:
fooman = PyFilesystemContentsManager.open_fs('s3://foo?endpoint_url=http://127.0.0.1:9000')
# fooman.get('')

## Set up a local PyFilesystemContentsManager

In [5]:
osman = PyFilesystemContentsManager.open_fs('osfs://%s' % os.getcwd())
# osman.get('')

## Create some dirs on our S3 filesystem and save this notebook into them

In [6]:
fooman.save(osman.get('pyfilesystem_s3fs.ipynb'), 'pyfilesystem_s3fs.ipynb')
fooman._save_directory('root0', None)
fooman._save_directory('root1', None)
fooman._save_directory('root1/leaf1', None)
fooman.save(osman.get("pyfilesystem_s3fs.ipynb"), 'root0/pyfilesystem_s3fs.ipynb')
fooman.save(osman.get("pyfilesystem_s3fs.ipynb"), 'root1/leaf1/pyfilesystem_s3fs.ipynb')

{'name': 'pyfilesystem_s3fs.ipynb',
 'path': 'root1/leaf1/pyfilesystem_s3fs.ipynb',
 'last_modified': datetime.datetime(2020, 3, 20, 7, 23, 33, tzinfo=<UTC>),
 'created': None,
 'content': None,
 'format': None,
 'mimetype': None,
 'size': 11229,
 'writable': True,
 'type': 'notebook'}

## Retrieve the saved data from our S3 filesystem

In [7]:
fpath = 'pyfilesystem_s3fs.ipynb'
fooman.get(fpath)

{'name': 'pyfilesystem_s3fs.ipynb',
 'path': 'pyfilesystem_s3fs.ipynb',
 'last_modified': datetime.datetime(2020, 3, 20, 7, 23, 33, tzinfo=<UTC>),
 'created': None,
 'content': {'cells': [{'cell_type': 'markdown',
    'metadata': {},
    'source': 'Launch local S3 via s3proxy in a docker container (will not persist data):\n\n```\ndocker run -p 9000:80 --env S3PROXY_AUTHORIZATION=none andrewgaul/s3proxy\n```'},
   {'cell_type': 'code',
    'execution_count': 1,
    'metadata': {'trusted': True},
    'outputs': [],
    'source': '%load_ext autoreload\n%autoreload 2'},
   {'cell_type': 'code',
    'execution_count': 2,
    'metadata': {'trusted': True},
    'outputs': [],
    'source': 'import boto3, botocore\nfrom fs import open_fs\nimport os\nimport sys\n\nfrom jupyterfs.pyfilesystem_manager import PyFilesystemContentsManager'},
   {'cell_type': 'markdown',
    'metadata': {},
    'source': '## Create a bucket to use as the filesystem'},
   {'cell_type': 'code',
    'execution_count': 1

In [8]:
fpath = 'root0/pyfilesystem_s3fs.ipynb'
fooman.get(fpath)

{'name': 'pyfilesystem_s3fs.ipynb',
 'path': 'root0/pyfilesystem_s3fs.ipynb',
 'last_modified': datetime.datetime(2020, 3, 20, 7, 23, 33, tzinfo=<UTC>),
 'created': None,
 'content': {'cells': [{'cell_type': 'markdown',
    'metadata': {},
    'source': 'Launch local S3 via s3proxy in a docker container (will not persist data):\n\n```\ndocker run -p 9000:80 --env S3PROXY_AUTHORIZATION=none andrewgaul/s3proxy\n```'},
   {'cell_type': 'code',
    'execution_count': 1,
    'metadata': {'trusted': True},
    'outputs': [],
    'source': '%load_ext autoreload\n%autoreload 2'},
   {'cell_type': 'code',
    'execution_count': 2,
    'metadata': {'trusted': True},
    'outputs': [],
    'source': 'import boto3, botocore\nfrom fs import open_fs\nimport os\nimport sys\n\nfrom jupyterfs.pyfilesystem_manager import PyFilesystemContentsManager'},
   {'cell_type': 'markdown',
    'metadata': {},
    'source': '## Create a bucket to use as the filesystem'},
   {'cell_type': 'code',
    'execution_cou

In [9]:
fpath = 'root1/leaf1/pyfilesystem_s3fs.ipynb'
fooman.get(fpath)

{'name': 'pyfilesystem_s3fs.ipynb',
 'path': 'root1/leaf1/pyfilesystem_s3fs.ipynb',
 'last_modified': datetime.datetime(2020, 3, 20, 7, 23, 33, tzinfo=<UTC>),
 'created': None,
 'content': {'cells': [{'cell_type': 'markdown',
    'metadata': {},
    'source': 'Launch local S3 via s3proxy in a docker container (will not persist data):\n\n```\ndocker run -p 9000:80 --env S3PROXY_AUTHORIZATION=none andrewgaul/s3proxy\n```'},
   {'cell_type': 'code',
    'execution_count': 1,
    'metadata': {'trusted': True},
    'outputs': [],
    'source': '%load_ext autoreload\n%autoreload 2'},
   {'cell_type': 'code',
    'execution_count': 2,
    'metadata': {'trusted': True},
    'outputs': [],
    'source': 'import boto3, botocore\nfrom fs import open_fs\nimport os\nimport sys\n\nfrom jupyterfs.pyfilesystem_manager import PyFilesystemContentsManager'},
   {'cell_type': 'markdown',
    'metadata': {},
    'source': '## Create a bucket to use as the filesystem'},
   {'cell_type': 'code',
    'executi

## scratch

### examine underlying fs-s3fs

In [ ]:
foofs = open_fs('s3://foo?endpoint_url=http://127.0.0.1:9000')
foofs.getinfo('').raw
foofs.getinfo('root1/')

In [ ]:
for fpath in ('pyfilesystem_s3fs.ipynb', 'root0/pyfilesystem_s3fs.ipynb', 'root1/leaf1/pyfilesystem_s3fs.ipynb'):
    print(foofs.getinfo(fpath).raw)

## Use only boto3 resource api (not client api)

In [24]:
bucket_name = 'foo'
boto_kwargs = dict(
    config=botocore.client.Config(signature_version=botocore.UNSIGNED),
    endpoint_url='http://127.0.0.1:9000',
)

resource = boto3.resource('s3', **boto_kwargs)

# check if bucket already exists
bucket = resource.Bucket(bucket_name)
bucket_exists = True
try:
    resource.meta.client.head_bucket(Bucket=bucket_name)
except botocore.exceptions.ClientError as e:
    # If it was a 404 error, then the bucket does not exist.
    error_code = e.response['Error']['Code']
    if error_code == '404':
        bucket_exists = False

if bucket_exists:
    # delete the bucket
    for key in bucket.objects.all():
        key.delete()
    bucket.delete()

resource.create_bucket(Bucket=bucket_name)

s3.Bucket(name='foo')

In [26]:
test_bucket = 'test'
test_endpoint_url = 'http://127.0.0.1:9000'

_boto_kw = dict(
    config=botocore.client.Config(signature_version=botocore.UNSIGNED),
    endpoint_url=test_endpoint_url,
)


def _s3Resource():
    return boto3.resource('s3', **_boto_kw)


def _s3CreateBucket(bucket_name):
    s3Resource = _s3Resource()

    # check if bucket already exists
    bucket = s3Resource.Bucket(bucket_name)
    bucket_exists = True
    try:
        s3Resource.meta.client.head_bucket(Bucket=bucket_name)
    except botocore.exceptions.ClientError as e:
        # If it was a 404 error, then the bucket does not exist.
        error_code = e.response['Error']['Code']
        if error_code == '404':
            bucket_exists = False

    if not bucket_exists:
        # create the bucket
        s3Resource.create_bucket(Bucket=bucket_name)


def _s3DeleteBucket(bucket_name):
    s3Resource = _s3Resource()

    # check if bucket already exists
    bucket = s3Resource.Bucket(bucket_name)
    bucket_exists = True
    try:
        s3Resource.meta.client.head_bucket(Bucket=bucket_name)
    except botocore.exceptions.ClientError as e:
        # If it was a 404 error, then the bucket does not exist.
        error_code = e.response['Error']['Code']
        if error_code == '404':
            bucket_exists = False

    if bucket_exists:
        # delete the bucket
        for key in bucket.objects.all():
            key.delete()
        bucket.delete()

# test idempotency
_s3CreateBucket(test_bucket)
_s3DeleteBucket(test_bucket)
_s3DeleteBucket(test_bucket)
_s3CreateBucket(test_bucket)
_s3CreateBucket(test_bucket)
_s3DeleteBucket(test_bucket)